In [1]:
import wandb
from main import main
import ml_collections as mlc
import logging
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from model import MInterface
from data import DInterface
from train_utils import load_logger, load_callbacks
from config.config import set_default_config
import yaml
import os

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
DEFAULT_CONFIG = mlc.ConfigDict()
set_default_config(DEFAULT_CONFIG)
with open('config/egnn_cfg_sweep.yaml', 'r') as f:
    default_arg = yaml.safe_load(f)
DEFAULT_CONFIG.update(default_arg)
DEFAULT_CONFIG.log_dir = f'./log/sweep/{DEFAULT_CONFIG.pl_module.model_name}/{DEFAULT_CONFIG.pl_data_module.dataset}'
if not os.path.exists(DEFAULT_CONFIG.log_dir):
    os.makedirs(DEFAULT_CONFIG.log_dir)


In [2]:

def sweep_main():
    logging.info('seed everything')
    pl.seed_everything(DEFAULT_CONFIG.seed)
    if True:
        logging.info('Using wandb')
        wandb.init(project=DEFAULT_CONFIG.project, group=DEFAULT_CONFIG.group, save_code=True, dir=DEFAULT_CONFIG.log_dir, reinit=True)
        args = DEFAULT_CONFIG
        args.update_from_flattened_dict(vars(wandb.config)['_items'])
        # raise Exception
        wandb.config.update(args.pl_module.model.to_dict())
        wandb.config.update(args.to_dict())
    logging.info('Loading data and model')
    data_module = DInterface(**args.pl_data_module)
    model = MInterface(**args.pl_module)

    logging.info('loading callbacks and logger')
    args.trainer.callbacks = load_callbacks(args)
    args.trainer.logger = load_logger(args)
    
    logging.info('creating trainer')
    trainer = Trainer(**args.trainer)
    
    if args.trainer.auto_scale_batch_size:
        logging.info('start auto scale batch size')
        trainer.tune(model, data_module)

    logging.info('start training')
    trainer.fit(model, data_module)
    
    logging.info('start testing')
    trainer.test(model, data_module)



In [3]:
wandb.login(key='local-8fe6e6b5840c4c05aaaf6aac5ca8c1fb58abbd1f', host='http://localhost:8080')


2023-03-15 22:07:20,610 - ERROR - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: huabei. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for localhost to your netrc file: /home/huabei/.netrc


True

In [4]:

# https://docs.wandb.ai/guides/sweeps/define-sweep-configuration
sweep_config = {
  "name" : "my-sweep",
  "method" : "random",
  "metric" : {'goal': 'minimize', 'name': 'val_loss'},
  "parameters" : {
    "pl_module.lr": {
      "distribution": "log_uniform_values",
      "min": 0.00005,
      "max": 0.005
    },
    # "pl_module.model.tanh": {
    #   "values": [True, False]
    # }
  }
}

sweep_id = wandb.sweep(sweep_config, project=DEFAULT_CONFIG.project)

Create sweep with ID: nw1xnlx2
Sweep URL: http://localhost:8080/huabei/egnn-sweep/sweeps/nw1xnlx2


In [5]:
# %%wandb

count = 15 # number of runs to execute
wandb.agent(sweep_id, function=sweep_main, count=count)

2023-03-15 22:07:28,662 - INFO - Starting sweep agent: entity=None, project=None, count=15
wandb: Agent Starting Run: cswkkyb7 with config:
wandb: 	pl_module.lr: 0.0010574768573226232
2023-03-15 22:07:28,844 - INFO - seed everything
Global seed set to 1234
2023-03-15 22:07:28,848 - INFO - Using wandb
2023-03-15 22:07:28,854 - ERROR - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
在工作区中该文件仍保持原有的换行符
在工作区中该文件仍保持原有的换行符


2023-03-15 22:07:38,454 - INFO - Loading data and model
2023-03-15 22:07:40,586 - INFO - loading callbacks and logger
/home/huabei/miniconda3/envs/pytorch112/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
2023-03-15 22:07:40,618 - INFO - creating trainer
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-03-15 22:07:40,766 - INFO - start training
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_mae            0.21898266673088074
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
global_step,▁▁▁
lr-Adam,███▇▂█▇▄▃▁██▇▆▅▄▃▂▂▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
test_mae,▁
test_r2,▁
train_loss,█▃▄▅▅▃▃▃▃▃▄▄▂▂▄▂▂▃▃▁▃▂▃▂▄▂▃▄▂▂▂▃▁▃▂▁▂▂▂▁
train_mae,█▅▄▄▃▃▃▃▂▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
train_r2,▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▆▅▆▃▃▃▃▃▂▃▃▃▂▂▂▂▁▁▁▂▁▂▂▁▁▃▁▁▂▁▁▁▁▁▁▁▁▁▁
val_mae,█▆▆▆▃▄▃▃▃▂▃▃▃▂▂▂▂▂▁▁▂▂▂▃▁▂▄▁▁▂▁▁▁▁▁▁▁▁▁▁


wandb: Agent Starting Run: z5hqm9se with config:
wandb: 	pl_module.lr: 0.00253837934061579
2023-03-15 22:34:03,274 - INFO - seed everything
Global seed set to 1234
2023-03-15 22:34:03,275 - INFO - Using wandb
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
在工作区中该文件仍保持原有的换行符
在工作区中该文件仍保持原有的换行符


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
global_step,▁▁▁
lr-Adam,███▇▂█▇▄▃▁██▇▆▅▄▃▂▂▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
test_mae,▁
test_r2,▁
train_loss,▃▁▃▂▂▂▁▂▁▁▆▅▂▂▃▂▁▂▂▁█▆▄▃▅▃▃▃▃▃▃▃▂▃▂▂▂▃▂▂
train_mae,▄▂▂▂▁▂▁▁▁▁█▃▂▂▂▂▁▁▁▁█▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂
train_r2,▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,▅▂▂▃▁▁▁▂▁▁▄▃▃▂▂▂▂▂▁▁█▆▅▅▅▄▄▃▄▃▃▃▂▂▂▂▂▂▂▂
val_mae,▆▃▂▃▂▂▁▂▁▁▅▃▃▂▂▂▂▂▂▁█▆▅▅▅▄▄▄▄▃▃▄▃▃▂▃▂▂▂▂


wandb: Agent Starting Run: lk0305ue with config:
wandb: 	pl_module.lr: 0.0010762859187571753
2023-03-16 04:17:29,185 - INFO - seed everything
Global seed set to 1234
2023-03-16 04:17:29,186 - INFO - Using wandb
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
在工作区中该文件仍保持原有的换行符
在工作区中该文件仍保持原有的换行符


2023-03-16 04:17:37,044 - INFO - Loading data and model
2023-03-16 04:17:38,828 - INFO - loading callbacks and logger
/home/huabei/miniconda3/envs/pytorch112/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning:

There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.

2023-03-16 04:17:38,832 - INFO - creating trainer
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-03-16 04:17:38,950 - INFO - start training
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precis

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_mae            0.21953141689300537
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
global_step,▁▁▁
lr-Adam,███▇▂█▇▄▃▁██▇▆▅▄▃▂▂▁████▇▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
test_mae,▁
test_r2,▁
train_loss,█▃▇▅▅▄▃▂▃▃▅▄▂▂▄▂▂▂▃▁▃▂▃▁▄▂▃▃▂▂▂▃▁▃▁▁▁▁▁▁
train_mae,█▅▄▄▃▃▃▃▂▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
train_r2,▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▄▃▃▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,█▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▁▁▂▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁
